<a href="https://colab.research.google.com/github/JLuceroVasquez/challenge-telecom-x-latam-parte-2/blob/main/TelecomX-notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Desafío Telecom parte 2**

In [2]:
#Para la manipulación de datos
import pandas as pd
import numpy as np

#Para visualización de datos y resultados
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance #Para calcular la importancia de permutación promedio de los features

#Para transformación de predictoras
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler #Normalización estándar para el modelo de Regresión logística

#Para división de datos en entrenamiento y evaluación, entrenamiento y optimización de modelos ML
from sklearn.model_selection import train_test_split #División de datos en conjunto de entrenamiento y prueba
from sklearn.model_selection import KFold, cross_validate #Entrenamiento y validación cruzada del Modelos ML
from sklearn.model_selection import GridSearchCV #Para optimización de los hiperparámetros de modelos ML

#Para el balanceo de categorías en la variable Y
from imblearn.over_sampling import SMOTE #Técnica de oversampling SMOTE
from imblearn.under_sampling import NearMiss #Técnica de undersampling NearMiss
from imblearn.combine import SMOTEENN #Técnica que combina el método oversampling SMOTE y undersampling ENN
from imblearn.pipeline import Pipeline as imbpipeline #Para crear un pipeline (secuencia de pasos) que permita entrenar un modelo con datos balanceados, y validarlo por separado con datos desbalanceados

#Modelos ML
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#Para la evaluación de modelos ML: Matriz de confusión
from sklearn.metrics import confusion_matrix #Para obtener los valores de la matriz de confusión
from sklearn.metrics import ConfusionMatrixDisplay #Para visualizar la matriz de confusión
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score #Para el cálculo de métricas que cuantifiquen el error tipo 1 y 2 de la matriz de confusión

#Para la evaluación de modelos ML: Curva ROC
from sklearn.metrics import RocCurveDisplay #Para visualizar la curva ROC

#Para la evaluación de modelos ML: Curva de precisión x sensibilidad
from sklearn.metrics import PrecisionRecallDisplay #Para visualizar la curva de precisión x sensibilidad

#Para la evaluación de modelos ML: Informe de clasificación
from sklearn.metrics import classification_report #Para visualizar las métricas de precisión, recall y F1 por cada categoría, y sus promedios

##**Preparación de los datos**

###*Extracción del archivo tratado*

In [3]:
'''
import chardet
with open('/content/TelecomX-data.csv', 'rb') as f:
  diccionario = chardet.detect(f.read())
  encode = diccionario['encoding']

print(encode)
'''

utf-8


In [4]:
url = 'https://github.com/JLuceroVasquez/challenge-telecom-x-latam-parte-2/raw/refs/heads/main/TelecomX-data.csv'
datos = pd.read_csv(url, encoding='utf-8')
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customerid             7043 non-null   object 
 1   churn                  7043 non-null   object 
 2   gender                 7043 non-null   object 
 3   seniorcitizen          7043 non-null   bool   
 4   partner                7043 non-null   bool   
 5   dependents             7043 non-null   bool   
 6   tenure                 7043 non-null   int64  
 7   phoneservice           7043 non-null   bool   
 8   multiplelines          7043 non-null   bool   
 9   internetservice        7043 non-null   object 
 10  onlinesecurity         7043 non-null   bool   
 11  onlinebackup           7043 non-null   bool   
 12  deviceprotection       7043 non-null   bool   
 13  techsupport            7043 non-null   bool   
 14  streamingtv            7043 non-null   bool   
 15  stre

###*Tipado de columnas*

In [5]:
#Convertir las columnas categóricas a tipo category
datos = datos.astype({col: 'category' for col in datos.select_dtypes([object, bool]).columns})
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   customerid             7043 non-null   category
 1   churn                  7043 non-null   category
 2   gender                 7043 non-null   category
 3   seniorcitizen          7043 non-null   category
 4   partner                7043 non-null   category
 5   dependents             7043 non-null   category
 6   tenure                 7043 non-null   int64   
 7   phoneservice           7043 non-null   category
 8   multiplelines          7043 non-null   category
 9   internetservice        7043 non-null   category
 10  onlinesecurity         7043 non-null   category
 11  onlinebackup           7043 non-null   category
 12  deviceprotection       7043 non-null   category
 13  techsupport            7043 non-null   category
 14  streamingtv            7043 non-null   c

###*Eliminación de Columnas Irrelevantes*
Se eliminaron las columnas `customerid` y `servicios_contratados`. Debido a que:
* **`customerid`**: No aporta información relevante para predecir la variable de respuesta churn.
* **`servicios_contratados`**: Posee información redundante que está implicita en otras columnas que detallan si el cliente tiene contratado o no un servicio en específico.

In [6]:
datos.drop(columns=['customerid', 'servicios_contratados'], inplace=True)
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   churn             7043 non-null   category
 1   gender            7043 non-null   category
 2   seniorcitizen     7043 non-null   category
 3   partner           7043 non-null   category
 4   dependents        7043 non-null   category
 5   tenure            7043 non-null   int64   
 6   phoneservice      7043 non-null   category
 7   multiplelines     7043 non-null   category
 8   internetservice   7043 non-null   category
 9   onlinesecurity    7043 non-null   category
 10  onlinebackup      7043 non-null   category
 11  deviceprotection  7043 non-null   category
 12  techsupport       7043 non-null   category
 13  streamingtv       7043 non-null   category
 14  streamingmovies   7043 non-null   category
 15  contract          7043 non-null   category
 16  paperlessbilling  7043 n

###*Eliminación de registro con datos nulos*
La columna `charges_total` era la única con datos nulos, un total de 11. Tenía dos opciones:
* Eliminar los 11 registros.
* Calcular el valor de los 11 registros a partir de la multiplicación de las variables `tenure x 12  x charges_monthly`.
Se eligió eliminar los 11 registros, pues aún quedaban 7032 registros para usar. Y además, no era seguro que la multiplicación produzca el valor correcto de `charges_total`. Puesto que a lo largo del tiempo que el cliente estaba en la empresa, el monto mensual cobrado podría haber variado.

In [7]:
#Conteo de registro con datos nulos
datos.isnull().sum()

,0
churn,0
gender,0
seniorcitizen,0
partner,0
dependents,0
tenure,0
phoneservice,0
multiplelines,0
internetservice,0
onlinesecurity,0


In [11]:
#Visualizamos los registros con datos nulos en la columna 'charges_total'
registros_nulos = datos[datos.isnull().any(axis=1)]
registros_nulos

,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total,cuenta_diaria
945,No,Femenino,False,True,True,0,False,False,DSL,True,...,True,True,True,False,Dos años,False,Tarjeta de crédito (automático),56.05,NaN,1.868333
1731,No,Femenino,False,True,True,0,True,False,No,False,...,False,False,False,False,Dos años,False,Cheque por correo,20.00,NaN,0.666667
1906,No,Masculino,False,False,True,0,True,True,DSL,True,...,False,True,False,False,Dos años,True,Transferencia bancaria (automático),61.90,NaN,2.063333
2025,No,Masculino,False,True,True,0,True,False,No,False,...,False,False,False,False,Un año,True,Cheque por correo,19.70,NaN,0.656667
2176,No,Masculino,False,False,True,0,True,False,No,False,...,False,False,False,False,Dos años,False,Cheque por correo,20.25,NaN,0.675000
2250,No,Masculino,False,True,True,0,True,True,No,False,...,False,False,False,False,Dos años,False,Cheque por correo,25.35,NaN,0.845000
2855,No,Femenino,False,True,True,0,True,True,DSL,False,...,True,True,True,False,Dos años,False,Cheque por correo,73.35,NaN,2.445000
3052,No,Masculino,False,True,True,0,True,True,No,False,...,False,False,False,False,Dos años,False,Cheque por correo,25.75,NaN,0.858333
3118,No,Femenino,False,True,True,0,False,False,DSL,True,...,True,True,True,False,Dos años,True,Transferencia bancaria (automático),52.55,NaN,1.751667
4054,No,Femenino,False,True,True,0,True,False,DSL,True,...,True,False,True,True,Dos años,False,Cheque por correo,80.85,NaN,2.695000


In [13]:
#Eliminamos los 11 registros con datos nulos en la columna 'charges_total'
datos.dropna(subset=['charges_total'], axis=0, how='any', inplace=True)
datos.isnull().sum()

,0
churn,0
gender,0
seniorcitizen,0
partner,0
dependents,0
tenure,0
phoneservice,0
multiplelines,0
internetservice,0
onlinesecurity,0


###*Encoding*
Antes de realizar la transformación OneHot, se verificó si las variables categóricas contenían demasiados valores únicos que debían unirse bajo una misma categoría. La única variable que parecía sospechosa fue `internetservice`, sin embargo, al revisar sus valores únicos me encontré conforme con las categorías y no realice ninguna corrección.

Luego, se realizó la transformación OneHot configurando el parámetro `drop='if_binary'` que elimina la primera categoría de la variable según el **orden interno** de categorías que maneja el encoder.

Ese orden se controla con el parámetro `categories`. Si está en "`auto`" (el valor por defecto), el encoder:

1. Detecta automáticamente las categorías.

2. Las ordena alfabéticamente (para str) o de menor a mayor (para int o bool). Entonces:

    * "femenino" → 0, "masculino" → 1.
    * False → 0, True → 1.

OneHotEncoder elimina una de ellas para evitar **colinealidad** (la famosa “trampa de las variables ficticias” en **regresión**). Esto es necesario, porque se empleará el modelo de regresión logística.

In [14]:
#Se verifica la cantidad de categorías en las variables tipo 'category'
datos.describe(include='category')

,churn,gender,seniorcitizen,partner,dependents,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod
count,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032
unique,2,2,2,2,2,2,2,3,2,2,2,2,2,2,3,2,4
top,No,Masculino,False,False,False,True,False,Fibra óptica,False,False,False,False,False,False,Mes a mes,True,Cheque electrónico
freq,5163,3549,5890,3639,4933,6352,4065,3096,5017,4607,4614,4992,4329,4301,3875,4168,2365


In [15]:
#Se visualizan los 3 valores únicos de la columna 'internetservice'
datos['internetservice'].unique()

['DSL', 'Fibra óptica', 'No']
Categories (3, object): ['DSL', 'Fibra óptica', 'No']

In [16]:
#Se crea una lista con el nombre de las columnas categóricas.
columnas_categoricas = datos.select_dtypes(include='category').columns
columnas_categoricas

Index(['churn', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod'],
      dtype='object')

In [17]:
#Se crea la regla para la transformación de columnas.
one_hot = make_column_transformer((OneHotEncoder(drop='if_binary', handle_unknown='ignore'),#En columna con dos categorias, se establece la primera como cero y la segunda como 1. Si se detectan categorías desconocidas durante la transformación, se ignoran y asigna un valor de cero.
                                   columnas_categoricas),
                                  remainder='passthrough',#Las columnas no nombradas en el transformador, se mantienen.
                                  sparse_threshold=0, #Proporción mínima de datos diferente de ceros que, de no superarse, genera una matriz sparse para optimizar la memoria y eficiencia computacional.
                                  force_int_remainder_cols=False) #Establecer en Falso mantiene los nombres de las columnas transformadas, en lugar de convertirlas en enteros.

#Se crea un array que contiene los datos de las variables explicativas transformadas.
datos_transformados = one_hot.fit_transform(datos)

#Se crea el dataframe con el array y nombre de las variables explicativas transformadas.
datos_transformados = pd.DataFrame(data= datos_transformados, columns=one_hot.get_feature_names_out())
datos_transformados.head()

,onehotencoder__churn_Si,onehotencoder__gender_Masculino,onehotencoder__seniorcitizen_1.0,onehotencoder__partner_1.0,onehotencoder__dependents_1.0,onehotencoder__phoneservice_1.0,onehotencoder__multiplelines_1.0,onehotencoder__internetservice_DSL,onehotencoder__internetservice_Fibra óptica,onehotencoder__internetservice_No,...,onehotencoder__contract_Un año,onehotencoder__paperlessbilling_1.0,onehotencoder__paymentmethod_Cheque electrónico,onehotencoder__paymentmethod_Cheque por correo,onehotencoder__paymentmethod_Tarjeta de crédito (automático),onehotencoder__paymentmethod_Transferencia bancaria (automático),remainder__tenure,remainder__charges_monthly,remainder__charges_total,remainder__cuenta_diaria
0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,9.0,65.6,593.30,2.186667
1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,9.0,59.9,542.40,1.996667
2,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,4.0,73.9,280.85,2.463333
3,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,13.0,98.0,1237.85,3.266667
4,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,3.0,83.9,267.40,2.796667


##**Correlación y selección de variables**

###*Análisis de correlación*

###*Análisis dirigido*

##**Modelo predictivo**

###*Separación de datos*

###*Creación de modelos*

###*Evaluación de modelos*

##**Optimización del mejor modelo**

###*Selección de variables según importancia*

###*Optimización de los hiperparámetros*

###*Exportación del modelo campeón*

##**Interpretación y conclusiones**
###*Análisis de la importancia de las variables*
Abc
###*Conclusión*
Abc